In [1]:
# PGHOST = 'haoma-db.c2ifgz0co6no.us-west-2.rds.amazonaws.com'
# PGDATABASE = 'haoma-db'
# PGUSER = 'postgres'
# PGPASSWORD = ''

%load_ext autoreload
%autoreload 2

In [2]:
from pipeline import *
from process_pdf import *

## Necessary Input from Discharge
1. Zipcode
2. bool_services: whether or not a patient needs
    - nursing
    - physical therapy
    - occuptational therapy
    - speech pathology
    - social services
    - home health aide
3. tier_weights = the amount of weight we want to add to each tier 
4. num_agencies = number of agencies to return

In [33]:
zipcode = 94080
bool_services = [True]*6 
tier_weights = {'star': 0, 'flagged': 2, 'ppr': 2, 'dtc': 1} 
num_agencies = 15
pdf_path = 'referral_example.pdf'

In [34]:
df_cal = load_df(zipcode)
flagged_qtopic = return_flagged(pdf_path, df_cal)

In [35]:
pipe_prep = Pipeline([('Drop Unnecessary Columns', Drop()),
                      ('Rename Columns', Rename()),
                      ('Filter Offered Services', FilterByService(bool_services)),
                      ('Impute Values', custom_imputer()),
                      ('Normalize/Reverse Asc. Columns', change_ascending_cols()),
                      ('Recommend', Recommend(tier_weights, flagged_qtopic, num_agencies))])

df, df_rec = pipe_prep.fit_transform(df_cal.copy())

In [36]:
df_rec

,name,ppr,dtc,breathing,skin_integrity,score
1185,21ST CENTURY HOME HEALTH SERVICES INC.,194.60,84.68,300.0,299.91,2311.19
1088,"BEST HOME HEALTH PROVIDERS, INC",191.96,84.38,279.9,300.00,2293.24
63,"ACCENTCARE HOME HEALTH OF CALIFORNIA, INC",194.72,77.89,269.1,300.00,2268.01
163,AMEDISYS HOME HEALTH CARE,195.02,79.72,296.1,300.00,2266.84
1028,CARE IN TOUCH HOME HEALTH AGENCY,197.80,83.33,278.4,299.55,2248.48
1026,HEALTHFLEX HOME HEALTH SERVICES,194.34,75.97,300.0,299.97,2234.78
575,HEALTH LINK HOME HEALTH AGENCY,194.88,75.57,263.4,300.00,2229.25
1165,"ADVANCED HEALTHCARE SERVICES, LLC",194.42,83.62,276.9,299.67,2224.21
251,"ASIAN NETWORK PACIFIC HOME CARE, INC",193.26,91.85,229.8,300.00,2208.81
181,PROFESSIONAL HOME CARE ASSOCIATES,196.98,92.21,259.5,299.55,2203.54
